# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [ ]:
import numpy as np
import requests as rq
import sys
np.set_printoptions(threshold=sys.maxsize)
# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [ ]:
challenge_id = 'belongfossildecrease' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

# Check the results
if query_results is None or len(query_results) != num_queries:
    raise ValueError("Query results are invalid or incomplete.")

print(query_results)

[ -8.  -1.  14. -22. -12. -11. -17.  30.  13.  -3.  14.   7.  -7.  -3.
 -20.   2. -39.  -0. -15. -14. -42.   4.  -3.  20.   6.  -3. -13. -20.
 -14.   1.   5.  -2.  -1.   5.  14. -15.   6. -12. -45.  -9.   9.  21.
 -54.  -8.   1. -24.   4. -33.  10.  -8. -11. -11.   1.   7. -10. -17.
 -22.  31.  24.  13. -19.  -1.  -6.   2.  -0.  12.  -2. -22.  -4.  -8.
 -10. -30.  20.  24. -15.  -9.  -0.  11.  16.  10.  24.  13. -16.   8.
  -9.  30.  -4.   2.  -8.   1.   8. -10.  -2.   4.  10.  26.   9.   7.
  43. -14. -15.  17. -26. -12. -12.  18. -13.  25.   7. -16.   6.  36.
   6. -12.  15. -18.  -1.   6. -14. -29. -15.  -4.   7.  17.   9. -14.
   7.  -4.  46.   9.   2.  -5.  -0.  -9.  15. -33.  -9.  -3. -35.  -5.
 -19.  26.   6.   5. -16. -15.  25.  -1. -12. -29.  30.  -9.  39.   6.
  -2.   3. -10.  -1. -12.  16.  22.   7.   2. -29.   9.  -2. -18. -20.
  22.  11.  -2.   1. -11.   8.  -7. -13. -10.  24.  33. -10.   3.  -9.
  67.  -9. -11.   6.  11.   9. -41.   2.  10.  -8.   5.   2. -34. -16.
  15. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [ ]:
from scipy.optimize import linprog

# Decision variables: x (the dataset) and e (absolute errors)
c = np.concatenate([np.zeros(n), np.ones(num_queries)])

# Constraints:
A_ub = np.block([
    [queries, -np.eye(num_queries)], # Handles upper bound constraint
    [-queries, -np.eye(num_queries)],  # Handles lower bound constraint
    [np.eye(n), np.zeros((n, num_queries))], # Enforces -1 <= x_j
    [-np.eye(n), np.zeros((n, num_queries))] # Enforces x_j <= 1
])

b_ub = np.concatenate([
    query_results,
    -query_results,
    np.ones(n),
    np.ones(n)
])

# Bounds for x in [-1, 1]
bounds = [(-1, 1)] * n + [(0, None)] * num_queries

# Solving the linear program
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

# Extracting the reconstructed dataset
x_reconstructed = np.sign(result.x[:n])

# Submitting the result
submission_result = query(challenge_id, x_reconstructed, submit=True)
# Printing the submission result
print(f"Reconstruction attack achieves fraction {(1 + submission_result / n) / 2} correct values")

Reconstruction attack achieves fraction 0.81 correct values
